# 한글-Claude-v2 Model: Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

---

# 0. 기본 지식

### 문맥
이전에는 모델이 타이어 교체 방법을 알려주는 것을 보았지만 관련 데이터를 수동으로 제공하고 컨텍스트를 직접 제공해야 했습니다. 우리는 Bedrock에서 사용 가능한 모델을 활용하고 교육 중에 배운 지식을 바탕으로 질문하고 수동 컨텍스트를 제공하는 접근 방식을 탐색했습니다. 이러한 접근 방식은 짧은 문서나 단일 톤 애플리케이션에서는 작동하지만, 모델에 전송된 프롬프트에 모두 들어맞을 수 없는 대규모 기업 문서가 있을 수 있는 기업 수준의 질문 응답으로 확장되지 않습니다.

### 패턴
RAG(Retreival Augmented Generation)라는 아키텍처를 구현하여 이 프로세스를 개선할 수 있습니다. RAG는 ​​언어 모델 외부(비모수적)에서 데이터를 검색하고 검색된 관련 데이터를 컨텍스트에 추가하여 프롬프트를 강화합니다.

이 노트에서는 질문 응답 패턴에 접근하여 문서를 찾고 활용하여 사용자 질문에 대한 답변을 제공하는 방법을 설명합니다.

### 챌린지
- 토큰 한도를 초과하는 대용량 문서 관리 방법
- 질문과 관련된 문서를 찾는 방법

### 제안
위의 과제에 대해 이 노트는 다음과 같은 전략을 제안합니다.
#### 도큐먼트 준비
![임베딩](./images/Embeddings_lang.png)

질문에 답하려면 먼저 문서를 처리하고 문서 저장소 인덱스에 저장해야 합니다.
- 문서를 로드
- 더 작은 덩어리(chunk)로 처리하고 분할합니다.
- Amazon Bedrock Titan Embeddings 모델을 사용하여 각 청크의 수치 벡터 표현 생성
- 청크와 해당 임베딩을 사용하여 인덱스 생성
#### Ask Question
![질문](./images/Chatbot_lang.png)

문서 색인이 준비되면 질문할 준비가 된 것이며, 질문한 질문에 따라 관련 문서를 가져옵니다. 다음 단계가 실행됩니다.
- 입력 질문의 임베딩 생성
- 질문 임베딩과 인덱스의 임베딩을 비교합니다.
- (상위 N) 관련 문서 청크를 가져옵니다.
- 프롬프트에서 컨텍스트의 일부로 해당 청크를 추가합니다.
- Amazon Bedrock의 모델에 프롬프트를 보냅니다.
- 검색된 문서를 기반으로 상황에 맞는 답변을 얻습니다.

## 사용 사례
#### 데이터세트
이 아키텍처 패턴을 설명하기 위해 "교보생명 건강보험 약관 문서를 사용합니다. 이 문서에서는 다음과 같은 주제를 설명합니다.
- 보험 보장 내용
- 보험 가입 안내
- 보험료 및 해지 환급금

#### 페르소나
보험 설계사가 질문에 답변을 제공하듯이 보험 내용을 상세히 이해하지 못하는 일반인의 페르소나를 가정해 보겠습니다.
모델은 문서를 바탕으로 쉬운 언어로 답변하려고 노력할 것입니다.

### 구현
RAG 접근 방식을 따르기 위해 이 노트북은 RAG와 같은 패턴을 효율적으로 구축할 수 있는 다양한 서비스 및 도구와 통합된 LangChain 프레임워크를 사용하고 있습니다. 우리는 다음 도구를 사용할 것입니다:

- **LLM(대형 언어 모델)**: Amazon Bedrock을 통해 사용 가능한 Anthropic Claude V2

  이 모델은 문서 덩어리(chunk)(chunk)를 이해하고 인간 친화적인 방식으로 답변을 제공하는 데 사용됩니다.
- **임베딩 모델**: Amazon Bedrock을 통해 사용 가능한 Amazon Titan 임베딩

  이 모델은 텍스트 문서의 수치 표현을 생성하는 데 사용됩니다.
- **문서 로더**: LangChain을 통해 사용 가능한 PDF 로더

  이는 소스에서 문서를 로드할 수 있는 로더입니다. 이 노트북을 위해 로컬 경로에서 샘플 파일을 로드합니다. 이는 기업 내부 시스템에서 문서를 로드하는 로더로 쉽게 대체될 수 있습니다.

- **Vector Store**: LangChain을 통해 FAISS 사용

  이 노트북에서는 이 메모리 내 벡터 저장소를 사용하여 임베딩과 문서를 모두 저장합니다. 기업 환경에서는 이는 AWS OpenSearch, pgVector가 포함된 RDS Postgres, ChromaDB, Pinecone 또는 Weaviate와 같은 영구 저장소로 대체될 수 있습니다.
- **색인**: 벡터인덱스

  인덱스는 입력 임베딩과 문서 임베딩을 비교하여 관련 문서를 찾는 데 도움이 됩니다.
- **래퍼**: 인덱스, 벡터 저장소, 임베딩 모델 및 LLM을 래핑하여 사용자의 논리를 추상화합니다.

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.

이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [FAISS](https://github.com/facebookresearch/faiss), 벡터 임베딩 저장
- [PyPDF](https://pypi.org/project/pypdf/), PDF 파일 처리용

In [22]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.249 "pypdf>=3.8,<4"

Note: you may need to restart the kernel to use updated packages.


# 1. Bedrock Client 생성

In [23]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
os.environ["BEDROCK_ENDPOINT_URL"] = "https://bedrock.us-east-1.amazonaws.com"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## 랭체인 구성

LLM과 Embeddings 모델을 인스턴스화하는 것부터 시작합니다. 여기서는 텍스트 생성을 위해 Anthropic Claude를 사용하고 텍스트 삽입을 위해 Amazon Titan을 사용합니다.

참고: Bedrock과 함께 사용 가능한 다른 모델을 선택할 수도 있습니다. 모델을 변경하려면 `model_id`를 다음과 같이 교체하면 됩니다.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

사용 가능한 모델 ID는 다음과 같습니다.

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v1

# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

In [24]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, 
              model_kwargs={'max_tokens_to_sample':1000})
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

In [26]:
llm, bedrock_embeddings

(Bedrock(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<botocore.client.Bedrock object at 0x7f9590f494e0>, region_name=None, credentials_profile_name=None, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1000}, endpoint_url=None),
 BedrockEmbeddings(client=<botocore.client.Bedrock object at 0x7f9590f494e0>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-e1t-medium', model_kwargs=None, endpoint_url=None))

# 3. 데이터 준비
먼저 문서 저장소를 구축하기 위해 이미 제공된 "교보 생명 건강보험 약관" 문서를 사용하겠습니다.

[LangChain에서 사용 가능한 PyPDF의 DirectoryLoader](https://python.langchain.com/en/latest/reference/modules/document_loaders.html)를 사용하여 문서를 로드하고 더 작은 덩어리(chunk)로 나눌 수 있습니다.

참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. 또한 임베딩 모델에는 입력 토큰 길이가 512개 토큰으로 제한되어 있으며 이는 한국어의 경우에 대략 180 ~ 200 글자로 변환됩니다. 이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 210자가 겹치는 약 50자의 청크를 생성합니다.

In [28]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data_kr/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 210,
    chunk_overlap  = 50,
#    separators = ['\n']
     # separators = ['\n','\n\n']    
)
docs = text_splitter.split_documents(documents)

In [29]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 9 documents loaded is 1184 characters.
After the split we have 66 documents more than the original 9.
Average length among 66 documents (after split) is 176 characters.


In [30]:
print("docs[0].page_content: \n", docs[0].page_content)

docs[0].page_content: 
 무배당  교보다이렉트
플러스건강보험  갱신형
KYOBO LIFE INSURANCE
이 상품안내장은 2021년 10월 기준으로 작성되었습니다.21.10 월
상품개정


In [31]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [-0.23535156 -0.02868652  0.02416992 ...  0.08251953 -0.15527344
  0.00427246]
Size of the embedding:  (4096,)


유사한 패턴에 따라 전체 코퍼스에 대해 임베딩을 생성하고 벡터 저장소에 저장할 수 있습니다.

이는 임베딩 모델과 문서를 입력받아 전체 벡터 저장소를 생성하는 LangChain 내부의 [FAISS](https://github.com/facebookresearch/faiss)  구현을 사용하여 쉽게 수행할 수 있습니다. [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores)를 사용하면 프롬프트 생성, 쿼리 임베딩 가져오기, 관련 문서 샘플링 및 LLM 호출과 같은 대부분의 무거운 작업을 추상화할 수 있습니다. [VectorStoreIndexWrapper](https://python.langchain.com/docs/modules/data_connection/retrievers)가 이를 도와줍니다.



# 4.FAISS 벡터 스토어 생성

**⚠️⚠️⚠️ 참고: 다음 셀을 실행하는 데 몇 분 정도 걸릴 수 있습니다 ⚠️⚠️⚠️**

In [32]:
%%time

from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

CPU times: user 198 ms, sys: 11.7 ms, total: 210 ms
Wall time: 10.2 s


In [33]:
wrapper_store_faiss

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7f95cc8de6e0>)

# 5.질문 및 답변

이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

In [34]:
query = "만기지급금만기 생존시 얼마 받아요?"


첫 번째 단계는 문서와 비교할 수 있도록 쿼리 임베딩을 만드는 것입니다.

In [35]:
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

array([-0.07324219, -0.01623535,  0.19335938, ...,  0.17578125,
        0.00063705,  0.07080078])

이 쿼리 임베딩을 사용하여 관련 문서를 가져올 수 있습니다.
이제 쿼리는 임베딩으로 표시되어 가장 관련성이 높은 정보를 제공하는 데이터 저장소에 대해 쿼리의 유사성 검색을 수행할 수 있습니다.

In [14]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}')
    print('---')

4 documents are fetched which are relevant to the query.
----
## Document 1: 일시금 및 월생활자금의 50% 지급
만기지급금만기 생존시
※ 최초계약(만기지급형)에 한함300만원
※ 보험기간 중 피보험자가 사망한 경우에는 사망 당시의 책임준비금을 계약자에게 지급하고 이 계약은 더 이상 효력을 가지지 않습니다.
※  보험료 납입기간 중 피보험자가 장해분류표 중 동일한 재해 또는 재해 이외의 동일한 원인으로 여러 신체부위의 장해지급률
---
## Document 2: 특약가입시 중증질환자[뇌혈관 및 심장질환] · 희귀질환자 산정특례대상으로
등록되었을 경우 경제적 부담을 덜고 제대로 치료 받을 수 있도록 보장합니다.상품특징보험료는 더 알뜰하게!
보장은 더 다양하게!
무배당  교보다이렉트플러스건강보험 (갱신형)  으로
한층 더 건강한 삶을 누려보세요
---
## Document 3: ※ 보험료 납입이 면제되었다 하더라도 갱신되는 경우 갱신계약의 보험료는 납입해야 함
아플 때 힘이 되도록 특정산정특례대상보장
특약가입시 중증질환자[뇌혈관 및 심장질환] · 희귀질환자 산정특례대상으로
등록되었을 경우 경제적 부담을 덜고 제대로 치료 받을 수 있도록 보장합니다.
---
## Document 4: 말기신부전증, 말기간질환, 말기폐질환) 진단시 주계약 보험료 납입기간 동안
차회 이후의 보험료를 내지 않고도 보장을 지속적으로 받을 수 있습니다.
※ 보험료 납입이 면제되었다 하더라도 갱신되는 경우 갱신계약의 보험료는 납입해야 함
아플 때 힘이 되도록 특정산정특례대상보장
특약가입시 중증질환자[뇌혈관 및 심장질환] · 희귀질환자 산정특례대상으로
---


이제 관련 문서가 준비되었으므로 LLM을 사용하여 이러한 문서를 기반으로 답변을 생성할 차례입니다.

유사성 검색 결과를 기반으로 검색된 관련 문서와 함께 초기 프롬프트를 사용합니다. 그런 다음 이를 결합하여 모델에 피드백하여 결과를 얻는 프롬프트를 생성합니다.

LangChain은 이를 쉽게 수행할 수 있는 방법에 대한 추상화를 제공합니다.

# 6. 빠른 방법
Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

In [37]:
query, llm

('만기지급금만기 생존시 얼마 받아요?',
 Bedrock(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<botocore.client.Bedrock object at 0x7f9590f494e0>, region_name=None, credentials_profile_name=None, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1000}, endpoint_url=None))

In [38]:
answer = wrapper_store_faiss.query(question=query, llm=llm)
print_ww(answer)

 일시금 및 월생활자금의 50% 지급
만기지급금만기 생존시
※ 최초계약(만기지급형)에 한함300만원
이 계약서에 따르면 만기 생존시 일시금 및 월생활자금의 50%인 300만원을 받습니다.


다른 질문을 해보죠

In [39]:
query_2 = "보험료 납입면제 관련 사항 알려 주세요"

In [40]:
answer_2 = wrapper_store_faiss.query(question=query_2, llm=llm)
print_ww(answer_2)

 네. 보험료 납입면제 관련하여 몇 가지 주요내용 안내드리겠습니다.

- 보험료 납입이 면제된 경우에도 갱신계약 보험료는 계속 납입하셔야 합니다.
- 보험료 납입기간 동안 보험료 납입면제 사유가 발생하면 그 이후 보험료 납입은 면제되나, 갱신 시에는 보험료를 계속 납입해야합니다.
- 즉, 보험료 납입면제는 현재 계약의 보험료 납입의무를 면제하는 것일 뿐, 갱신계약의 보험료 납입의무까지 면제하는 것은 아닙니다.

이러한 내용을 참고하시어 보험료 납입 면제와 갱신계약 보험료 납입에 대해 혼동하지 않도록 유의하시기 바랍니다. 도움이 되셨길 바랍니다.


#### 보험 약관에 없는 내용 질문

In [41]:
query_3 = "치과 보장 내역 알려 주세요?"
answer_3 = wrapper_store_faiss.query(question=query_3, llm=llm)
print_ww(answer_3)

 제공된 정보에서 치과 보장과 관련한 내용을 찾을 수 없습니다. 교보다이렉트 플러스건강보험 상품의 정확한 치과 보장 내역은 확인하기 어렵습니다. 죄송합니다만 치과 보장 관련 정보가
없어 자세히 알려드리기 어렵습니다.


# 7.사용자 정의 가능한 옵션
위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

In [42]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: 다음 문맥을 사용하여 마지막 질문에 대한 간결한 답변을 제공하세요. 답을 모르면 모른다고 말하고 답을 만들어내려고 하지 마세요.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 20}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# query = "3년 경과 후에 해지환급금 얼마 받아요?"  
query = "5대 질병 진단시 지급되는 금액은 얼마인가요?"
result = qa({"query": query})
print_ww(result['result'])

 주어진 문맥에 따르면 5대 질병(뇌출혈, 급성심근경색증, 말기신부전증, 말기간질환, 말기폐질환) 진단 시 다음과 같이 지급됩니다:

- 일시금 1,500만원
- 월생활자금 50만원 × 36개월

총 지급금액은 1,800만원입니다.

다만 최초 계약 가입 후 1년 미만 진단확정 시 일시금 및 월생활자금의 50%가 지급됩니다.


In [44]:
def show_context_used(context_list):
    for idx, context in enumerate(context_list):
        print("-----------------------------------------------")                
        print(f"{idx+1}. context to be fed into FM(e.g. Claude-v2)")
        print("-----------------------------------------------")        
        print_ww(context.page_content)        
        
show_context_used(result['source_documents'])        

-----------------------------------------------
1. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
일시금 및 월생활자금의 50% 지급
만기지급금만기 생존시
※ 최초계약(만기지급형)에 한함300만원
※ 보험기간 중 피보험자가 사망한 경우에는 사망 당시의 책임준비금을 계약자에게 지급하고 이 계약은 더 이상 효력을 가지지 않습니다.
※  보험료 납입기간 중 피보험자가 장해분류표 중 동일한 재해 또는 재해 이외의 동일한 원인으로 여러 신체부위의 장해지급률
-----------------------------------------------
2. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
말기폐질환으로 진단확정시
※ 다만, 최초 1회의 진단확정에 한함
※ 갱신계약의 경우 [말기폐질환 보장계약]
     으로 갱신된 경우에 한함일시금  1,500 만원
+
월생활자금  50만원 × 36개월 확정지급
(총지급금액 1,800만원)
※ 다만, 최초 계약은 가입 후 1년 미만 진단확정시
     일시금 및 월생활자금의 50% 지급
만기지급금만기 생존시
-----------------------------------------------
3. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
을 더하여 50% 이상 장해상태가 되었을 경우 또는 뇌출혈, 급성심근경색증, 말기신부전증, 말기간질환 또는 말기폐질환으로
진단이 확정된 경우에는 차회 이후의 보험료 납입을 면제합니다. 그러나 보험료의 납입이 면제되었다 하더라도 갱신시 갱신
계약의 보험료는 납입하여야 하며, 최초계약의 보장개시일 이후 이미 보험료의 납입

# 8.결론
RAG(검색 증강 생성)에 관한 이 모듈을 완료한 것을 축하합니다! 이는 대규모 언어 모델의 성능과 검색 방법의 정확성을 결합하는 중요한 기술입니다. 검색된 관련 사례로 생성을 강화함으로써 우리가 받은 응답은 더욱 일관되고 근거가 있게 됩니다. 당신은 이 혁신적인 접근 방식을 배우는 것에 자부심을 느껴야 합니다. 나는 당신이 얻은 지식이 창의적이고 매력적인 언어 생성 시스템을 구축하는 데 매우 유용할 것이라고 확신합니다. 잘하셨어요!

위의 RAG 기반 질문 응답 구현에서 우리는 다음 개념과 Amazon Bedrock 및 LangChain 통합을 사용하여 이를 구현하는 방법을 탐색했습니다.

- 문서 로드 및 임베딩 생성을 통해 벡터 저장소 생성
- 질문에 대한 문서 검색
- LLM에 대한 입력으로 사용되는 프롬프트 준비
- 인간 친화적인 방식으로 답변 제시

### 요약
- 다양한 벡터 스토어로 실험해 보세요.
- Amazon Bedrock에서 사용 가능한 다양한 모델을 활용하여 대체 출력을 확인하세요.
- 임베딩 및 문서 청크의 영구 저장과 같은 옵션 탐색
- 엔터프라이즈 데이터 저장소와의 통합

